In [1]:
import os
import git
from pathlib import Path
from scipy.spatial import Delaunay

ROOT_DIR =  Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
SAVE_FIGS = False

In [2]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
from reporting import *
plots_path = os.path.join(ROOT_DIR, "publication", "paper", "draft_plots")
main_df = main_df.copy()
np.random.seed(0)

In [ ]:
main_df = main_df.fillna("None", inplace=False)

In [25]:
def in_hull(p, hull):
    if hasattr(hull, 'vertices') and not isinstance(hull, Delaunay):
        hull = Delaunay(hull.points)
    elif not isinstance(hull, Delaunay):
        hull = Delaunay(hull)
    return hull.find_simplex(p) >= 0

x_vals = np.linspace(0, 20, 1000)
eta_vals = 1.5 + np.zeros_like(x_vals)
roi_beta = (eta_vals) / x_vals
roi = 1 / roi_beta
def line_intersects_hull(hull):
    if hull is None or hull is np.nan:
        return False
    line_points = np.column_stack((x_vals, roi))
    return np.any(in_hull(line_points, hull))

main_df['intersect_roi'] = main_df['hull'].apply(lambda h: line_intersects_hull(h))
main_df


C:\Users\yashd\AppData\Local\Temp\ipykernel_45960\641257423.py:10: RuntimeWarning: divide by zero encountered in divide
  roi_beta = (eta_vals) / x_vals


,group,obs_var,var_lower,var_upper,obs_kurt,kurt_lower,kurt_upper,total_samples,initial_r,initial_eta,...,dataset_type,hull,best_beta,best_1/beta,beat_all_priors,best_prior,failure_category,failure_type,which_ones,intersect_roi
0,2,1292.946700,749.107670,2108.430700,421.841550,123.974430,696.238040,45000.0,0.1,2.5,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,38.000000,0.026316,1,GenGamma,practically_pass,pass,NaN,True
1,5,662.654400,382.089750,1248.617800,628.030000,108.212240,1663.239500,117000.0,0.1,2.0,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,43.333333,0.023077,1,GenGamma,practically_pass,pass,NaN,False
2,8,256.633900,133.792040,650.845600,1363.730700,99.365600,3639.235800,189000.0,0.1,2.9,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,53.333333,0.018750,1,GenGamma,practically_pass,pass,NaN,True
3,11,121.961570,57.215298,390.192930,2698.317600,93.680930,4889.278000,495000.0,0.1,3.1,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,55.555556,0.018000,1,GenGamma,practically_pass,pass,NaN,True
4,14,48.468010,23.179575,151.679550,2654.295200,86.305040,4711.629400,1134000.0,0.1,3.4,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,56.666667,0.017647,1,GenGamma,practically_pass,pass,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,4,241418.060000,225806.880000,257381.550000,4.450259,4.034465,4.879806,22538.0,0.5,-0.5,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x0...,1.400000,0.714286,1,GenGamma,actually_pass,pass,NaN,False
1151,5,33947.695000,31871.596000,36147.625000,3.966855,3.001096,5.099455,296776.0,0.1,8.6,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x0...,101.000000,0.009901,1,GenGamma,actually_pass,pass,NaN,True
1152,6,6416.669400,5792.756300,7090.980500,13.477583,10.768813,16.435010,2880648.0,0.1,4.2,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x0...,56.000000,0.017857,1,GenGamma,practically_pass,pass,NaN,True
1153,7,397.892600,357.501070,450.380680,17.566162,9.929181,54.616486,25362406.0,0.1,2.2,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x0...,37.000000,0.027027,1,GenGamma,practically_pass,pass,NaN,True


In [32]:
mainDFGrouped = main_df.copy()
mainDFGrouped = mainDFGrouped[mainDFGrouped["failure_type"]!= "low samples"]

mainDFGrouped["pass"] = mainDFGrouped["failure_category"].apply(lambda x: x in ["practically_pass", "actually_pass"])
mainDFGrouped = mainDFGrouped[["dataset_type","dataset", "subset", "transform", "orientation", "failure_category", "pass", "beat_all_priors", "best_prior", "intersect_roi"]]
mainDFGrouped.fillna("None", inplace=True)
mainDFGrouped["number"] = (
    mainDFGrouped.groupby(["dataset_type", "dataset", "subset", "transform", "failure_category"])["pass"]
    .transform("count")
)

mainDFGrouped["pass_percentage"] = np.round(
    mainDFGrouped.groupby(["dataset_type", "dataset", "subset", "transform"])["pass"]
    .transform("mean") * 100
)

mainDFGrouped["beat_all_priors_percentage"] = np.round(
    mainDFGrouped.groupby(["dataset_type", "dataset", "subset", "transform"])["beat_all_priors"]
    .transform("mean") * 100
)

mainDFGrouped["percent_intersect_roi"] = np.round(
    mainDFGrouped.groupby(["dataset_type", "dataset", "subset", "transform"])["intersect_roi"]
    .transform("mean") * 100
)


mainDFGrouped = mainDFGrouped.groupby(["dataset_type", "dataset", "subset", "transform", "failure_category"]).first().reset_index().sort_values(by=["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category", "intersect_roi"])
mainDFGrouped.to_csv("results.csv")


In [33]:
mainDFGrouped = main_df.copy()
mainDFGrouped = mainDFGrouped[mainDFGrouped["failure_type"]!= "low samples"]

mainDFGrouped["pass"] = mainDFGrouped["failure_category"].apply(lambda x: x in ["practically_pass", "actually_pass"])
mainDFGrouped = mainDFGrouped[["dataset_type","dataset", "subset", "transform", "orientation", "failure_category", "pass", "beat_all_priors", "best_prior"]]
mainDFGrouped.fillna("None", inplace=True)
mainDFGrouped["number"] = (
    mainDFGrouped.groupby(["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"])["pass"]
    .transform("count")
)

mainDFGrouped["pass_percentage"] = np.round(
    mainDFGrouped.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"])["pass"]
    .transform("mean") * 100
)

mainDFGrouped["beat_all_priors_percentage"] = np.round(
    mainDFGrouped.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"])["beat_all_priors"]
    .transform("mean") * 100
)


mainDFGrouped = mainDFGrouped.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first().reset_index().sort_values(by=["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"])
mainDFGrouped


,dataset_type,dataset,subset,transform,orientation,failure_category,pass,beat_all_priors,best_prior,number,pass_percentage,beat_all_priors_percentage
0,medical,syntheticMRI2D,axial,wavelet,diagonal,trivial_failure,False,0,Gaussian,1,86.0,71.0
1,medical,syntheticMRI2D,axial,wavelet,horizontal,trivial_failure,False,0,Gaussian,2,71.0,86.0
2,medical,syntheticMRI2D,axial,wavelet,vertical,trivial_failure,False,0,Gaussian,2,71.0,71.0
3,medical,syntheticMRI2D,coronal,wavelet,diagonal,actually_pass,True,1,GenGamma,2,100.0,86.0
4,medical,syntheticMRI2D,coronal,wavelet,horizontal,trivial_failure,False,0,Gaussian,1,71.0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,remote sensing,spaceNet,full,learned,misc,trivial_failure,False,1,GenGamma,2,50.0,100.0
73,remote sensing,spaceNet,full,learned,multi_edge,practically_pass,True,1,GenGamma,10,100.0,100.0
74,remote sensing,spaceNet,full,learned,single_edge,practically_pass,True,1,GenGamma,10,50.0,100.0
75,remote sensing,spaceNet,full,wavelet,diagonal,practically_pass,True,1,GenGamma,17,62.0,91.0


In [ ]:
ordered_failcat_cols = ["actually_pass", "practically_pass", "borderline", "interesting_failure", "trivial_failure"]
ordered_prior_cols = ["GenGamma", "Gaussian", "Laplace", "Student-T"]

# Remote Sensing

In [ ]:
DATASET_TYPE = "remote sensing" 

In [ ]:
medical = mainDFGrouped.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical = medical.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first()[["pass_percentage", "beat_all_priors_percentage"]].reset_index()
medical

,dataset_type,dataset,subset,transform,orientation,pass_percentage,beat_all_priors_percentage
0,remote sensing,agriVision,full,fourier,None,95.0,98.0
1,remote sensing,agriVision,full,learned,dual_color,100.0,100.0
2,remote sensing,agriVision,full,learned,eye,17.0,67.0
3,remote sensing,agriVision,full,learned,inside_out,43.0,57.0
4,remote sensing,agriVision,full,learned,misc,50.0,75.0
5,remote sensing,agriVision,full,learned,multi_edge,80.0,100.0
6,remote sensing,agriVision,full,learned,single_edge,100.0,100.0
7,remote sensing,agriVision,full,wavelet,diagonal,100.0,100.0
8,remote sensing,agriVision,full,wavelet,horizVert,100.0,100.0
9,remote sensing,pastis,full,fourier,None,100.0,100.0


In [ ]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="failure_category",
    values="number",
    aggfunc="sum",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
# Reorder columns if present in medical_pivot_percent
ordered_cols_present = [col for col in ordered_failcat_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent


failure_category                                        actually_pass  \
dataset_type   dataset    subset transform orientation                  
remote sensing agriVision full   fourier   None                   2.0   
                                 learned   dual_color             0.0   
                                           eye                    0.0   
                                           inside_out             0.0   
                                           misc                   0.0   
                                           multi_edge             0.0   
                                           single_edge            0.0   
                                 wavelet   diagonal              17.0   
                                           horizVert             11.0   
               pastis     full   fourier   None                  91.0   
                                 learned   dual_color             0.0   
                                           misc                   0.0   
                                           multi_edge             0.0   
                                           single_edge            0.0   
                                 wavelet   diagonal              57.0   
                                           horizVert             57.0   
               spaceNet   full   fourier   None                   0.0   
                                 learned   dual_color             0.0   
                                           eye                    0.0   
                                           inside_out             0.0   
                                           misc                   0.0   
                                           multi_edge             0.0   
                                           single_edge            0.0   
                                 wavelet   diagonal               9.0   
                                           horizVert              0.0   

failure_category                                        practically_pass  \
dataset_type   dataset    subset transform orientation                     
remote sensing agriVision full   fourier   None                     93.0   
                                 learned   dual_color              100.0   
                                           eye                      17.0   
                                           inside_out               43.0   
                                           misc                     50.0   
                                           multi_edge               80.0   
                                           single_edge             100.0   
                                 wavelet   diagonal                 83.0   
                                           horizVert                89.0   
               pastis     full   fourier   None                      9.0   
                                 learned   dual_color               91.0   
                                           misc                      0.0   
                                           multi_edge              100.0   
                                           single_edge              83.0   
                                 wavelet   diagonal                 43.0   
                                           horizVert                43.0   
               spaceNet   full   fourier   None                    100.0   
                                 learned   dual_color               15.0   
                                           eye                       0.0   
                                           inside_out                0.0   
                                           misc                     50.0   
                                           multi_edge              100.0   
                                           single_edge              50.0   
                                 wavelet   diagonal                 53.0   
                                           horizVert

In [ ]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                              GenGamma  Gaussian  \
dataset_type   dataset    subset transform orientation                       
remote sensing agriVision full   fourier   None             98.0       0.0   
                                 learned   dual_color      100.0       0.0   
                                           eye              67.0       0.0   
                                           inside_out       57.0       0.0   
                                           misc             75.0       0.0   
                                           multi_edge      100.0       0.0   
                                           single_edge     100.0       0.0   
                                 wavelet   diagonal        100.0       0.0   
                                           horizVert       100.0       0.0   
               pastis     full   fourier   None            100.0       0.0   
                                 learned   dual_color      100.0       0.0   
                                           misc             50.0      50.0   
                                           multi_edge      100.0       0.0   
                                           single_edge     100.0       0.0   
                                 wavelet   diagonal        100.0       0.0   
                                           horizVert        96.0       0.0   
               spaceNet   full   fourier   None            100.0       0.0   
                                 learned   dual_color      100.0       0.0   
                                           eye              75.0       0.0   
                                           inside_out       60.0       0.0   
                                           misc            100.0       0.0   
                                           multi_edge      100.0       0.0   
                                           single_edge     100.0       0.0   
                                 wavelet   diagonal         91.0       9.0   
                                           horizVert       100.0       0.0   

best_prior                                              Laplace  Student-T  
dataset_type   dataset    subset transform orientation                      
remote sensing agriVision full   fourier   None             0.0        2.0  
                                 learned   dual_color       0.0        0.0  
                                           eye              0.0       33.0  
                                           inside_out       0.0       43.0  
                                           misc             0.0       25.0  
                                           multi_edge       0.0        0.0  
                                           single_edge      0.0        0.0  
                                 wavelet   diagonal         0.0        0.0  
                                           horizVert        0.0        0.0  
               pastis     full   fourier   None             0.0        0.0  
                                 learned   dual_color       0.0        0.0  
                                           misc             0.0        0.0  
                                           multi_edge       0.0        0.0  
                                           single_edge      0.0        0.0  
                                 wavelet   diagonal         0.0        0.0  
                                           horizVert        0.0        4.0  
               spaceNet   full   fourier   None             0.0        0.0  
                                 learned   dual_color       0.0        0.0  
                                           eye             25.0        0.0  
                                           inside_out       0.0       40.0  
                                           misc             0.0        0.0  
                                           multi_edge       0.0        0.0  
                                           si

In [ ]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "failure_category"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                      GenGamma  \
dataset_type   dataset    subset transform failure_category                
remote sensing agriVision full   fourier   actually_pass           100.0   
                                           interesting_failure     100.0   
                                           practically_pass         98.0   
                                 learned   borderline                0.0   
                                           interesting_failure     100.0   
                                           practically_pass        100.0   
                                           trivial_failure          50.0   
                                 wavelet   actually_pass           100.0   
                                           practically_pass        100.0   
               pastis     full   fourier   actually_pass           100.0   
                                           practically_pass        100.0   
                                 learned   borderline              100.0   
                                           practically_pass        100.0   
                                           trivial_failure          50.0   
                                 wavelet   actually_pass            97.0   
                                           practically_pass        100.0   
               spaceNet   full   fourier   practically_pass        100.0   
                                 learned   borderline              100.0   
                                           interesting_failure     100.0   
                                           practically_pass        100.0   
                                           trivial_failure          83.0   
                                 wavelet   actually_pass           100.0   
                                           borderline              100.0   
                                           practically_pass        100.0   
                                           trivial_failure          88.0   

best_prior                                                      Gaussian  \
dataset_type   dataset    subset transform failure_category                
remote sensing agriVision full   fourier   actually_pass             0.0   
                                           interesting_failure       0.0   
                                           practically_pass          0.0   
                                 learned   borderline                0.0   
                                           interesting_failure       0.0   
                                           practically_pass          0.0   
                                           trivial_failure           0.0   
                                 wavelet   actually_pass             0.0   
                                           practically_pass          0.0   
               pastis     full   fourier   actually_pass             0.0   
                                           practically_pass          0.0   
                                 learned   borderline                0.0   
                                           practically_pass          0.0   
                                           trivial_failure          50.0   
                                 wavelet   actually_pass             0.0   
                                           practically_pass          0.0   
               spaceNet   full   fourier   practically_pass          0.0   
                                 learned   borderline                0.0   
                                           interesting_failure       0.0   
                                           practically_pass          0.0   
                                           trivial_failure           0.0   
                                 wavelet   actually_pass             0.0   
                                           borderline                0.0   
                                           prac

# Natural Images

In [ ]:
DATASET_TYPE = "natural" 

In [ ]:
medical = mainDFGrouped.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical = medical.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first()[["pass_percentage", "beat_all_priors_percentage"]].reset_index()
medical

,dataset_type,dataset,subset,transform,orientation,pass_percentage,beat_all_priors_percentage
0,natural,coco,indoor,learned,dual_color,46.0,100.0
1,natural,coco,indoor,learned,eye,0.0,83.0
2,natural,coco,indoor,learned,inside_out,12.0,50.0
3,natural,coco,indoor,learned,misc,40.0,60.0
4,natural,coco,indoor,learned,multi_edge,90.0,100.0
5,natural,coco,indoor,learned,single_edge,80.0,100.0
6,natural,coco,indoor,wavelet,diagonal,62.0,100.0
7,natural,coco,indoor,wavelet,horizontal,9.0,81.0
8,natural,coco,indoor,wavelet,vertical,75.0,94.0
9,natural,coco,outdoor,learned,dual_color,77.0,100.0


In [ ]:

medical = mainDFGrouped.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="failure_category",
    values="number",
    aggfunc="sum",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
# Reorder columns if present in medical_pivot_percent
ordered_cols_present = [col for col in ordered_failcat_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent


failure_category                                            actually_pass  \
dataset_type dataset         subset  transform orientation                  
natural      coco            indoor  learned   dual_color             0.0   
                                               eye                    0.0   
                                               inside_out             0.0   
                                               misc                   0.0   
                                               multi_edge             0.0   
                                               single_edge            0.0   
                                     wavelet   diagonal              38.0   
                                               horizontal             0.0   
                                               vertical              38.0   
                             outdoor learned   dual_color             0.0   
                                               eye                    0.0   
                                               inside_out             0.0   
                                               misc                   0.0   
                                               multi_edge             0.0   
                                               single_edge            0.0   
                                     wavelet   diagonal              38.0   
                                               horizontal             0.0   
                                               vertical              38.0   
             segmentAnything full    learned   dual_color             0.0   
                                               eye                    0.0   
                                               inside_out             0.0   
                                               misc                   0.0   
                                               multi_edge             0.0   
                                               single_edge            0.0   
                                     wavelet   diagonal              42.0   
                                               horizontal             0.0   
                                               vertical              33.0   
             standardTesting full    learned   dual_color             0.0   
                                               eye                    0.0   
                                               inside_out             0.0   
                                               misc                   0.0   
                                               multi_edge             0.0   
                                               single_edge            0.0   
                                     wavelet   diagonal               0.0   
                                               horizontal             0.0   
                                               vertical               0.0   

failure_category                                            practically_pass  \
dataset_type dataset         subset  transform orientation                     
natural      coco            indoor  learned   dual_color               46.0   
                                               eye                       0.0   
                                               inside_out               12.0   
                                               misc                     40.0   
                                               multi_edge               90.0   
                                               single_edge              80.0   
                                     wavelet   diagonal                 25.0   
                                               horizontal                9.0   
                                               vertical                 38.0   
                             outdoor learned   dual_color               77.0   
                                               eye                       0.0   
                                 

In [ ]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                  GenGamma  \
dataset_type dataset         subset  transform orientation             
natural      coco            indoor  learned   dual_color      100.0   
                                               eye              83.0   
                                               inside_out       50.0   
                                               misc             60.0   
                                               multi_edge      100.0   
                                               single_edge     100.0   
                                     wavelet   diagonal        100.0   
                                               horizontal       81.0   
                                               vertical         94.0   
                             outdoor learned   dual_color      100.0   
                                               eye             100.0   
                                               inside_out       56.0   
                                               misc             83.0   
                                               multi_edge      100.0   
                                               single_edge     100.0   
                                     wavelet   diagonal        100.0   
                                               horizontal       84.0   
                                               vertical        100.0   
             segmentAnything full    learned   dual_color      100.0   
                                               eye             100.0   
                                               inside_out       44.0   
                                               misc             67.0   
                                               multi_edge      100.0   
                                               single_edge     100.0   
                                     wavelet   diagonal        100.0   
                                               horizontal       86.0   
                                               vertical        100.0   
             standardTesting full    learned   dual_color      100.0   
                                               eye              75.0   
                                               inside_out       67.0   
                                               misc             75.0   
                                               multi_edge      100.0   
                                               single_edge     100.0   
                                     wavelet   diagonal         78.0   
                                               horizontal       78.0   
                                               vertical         78.0   

best_prior                                                  Gaussian  Laplace  \
dataset_type dataset         subset  transform orientation                      
natural      coco            indoor  learned   dual_color        0.0      0.0   
                                               eye               0.0      0.0   
                                               inside_out       38.0      0.0   
                                               misc              0.0      0.0   
                                               multi_edge        0.0      0.0   
                                               single_edge       0.0      0.0   
                                     wavelet   diagonal          0.0      0.0   
                                               horizontal       12.0      6.0   
                                               vertical          6.0      0.0   
                             outdoor learned   dual_color        0.0      0.0   
                                               eye               0.0      0.0   
                                               inside_out       33.0     11.0   
                                               misc             17.0      0.0   
                                               m

In [ ]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "failure_category"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                          GenGamma  \
dataset_type dataset         subset  transform failure_category                
natural      coco            indoor  learned   borderline              100.0   
                                               interesting_failure     100.0   
                                               practically_pass         97.0   
                                               trivial_failure          54.0   
                                     wavelet   actually_pass            92.0   
                                               borderline              100.0   
                                               interesting_failure     100.0   
                                               practically_pass        100.0   
                                               trivial_failure          70.0   
                             outdoor learned   borderline              100.0   
                                               interesting_failure      50.0   
                                               practically_pass        100.0   
                                               trivial_failure          67.0   
                                     wavelet   actually_pass           100.0   
                                               borderline              100.0   
                                               interesting_failure     100.0   
                                               practically_pass        100.0   
                                               trivial_failure          74.0   
             segmentAnything full    learned   borderline              100.0   
                                               interesting_failure      75.0   
                                               practically_pass        100.0   
                                               trivial_failure          50.0   
                                     wavelet   actually_pass           100.0   
                                               practically_pass        100.0   
                                               trivial_failure          69.0   
             standardTesting full    learned   None                     94.0   
                                     wavelet   None                     78.0   

best_prior                                                          Gaussian  \
dataset_type dataset         subset  transform failure_category                
natural      coco            indoor  learned   borderline                0.0   
                                               interesting_failure       0.0   
                                               practically_pass          0.0   
                                               trivial_failure          23.0   
                                     wavelet   actually_pass             8.0   
                                               borderline                0.0   
                                               interesting_failure       0.0   
                                               practically_pass          0.0   
                                               trivial_failure          20.0   
                             outdoor learned   borderline                0.0   
                                               interesting_failure       0.0   
                                               practically_pass          0.0   
                                               trivial_failure          33.0   
                                     wavelet   actually_pass             0.0   
                                               borderline                0.0   
                                               interesting_failure       0.0   
                                               practically_pass          0.0   
                                               trivial_failure          11.0   
             segmentAnything full    learned   borderline                0.0   

# Medical

In [ ]:
DATASET_TYPE = "medical"

In [ ]:
medical = mainDFGrouped.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical = medical.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first()[["pass_percentage", "beat_all_priors_percentage"]].reset_index()
medical

,dataset_type,dataset,subset,transform,orientation,pass_percentage,beat_all_priors_percentage
0,medical,syntheticMRI2D,axial,wavelet,diagonal,86.0,71.0
1,medical,syntheticMRI2D,axial,wavelet,horizontal,71.0,86.0
2,medical,syntheticMRI2D,axial,wavelet,vertical,71.0,71.0
3,medical,syntheticMRI2D,coronal,wavelet,diagonal,100.0,86.0
4,medical,syntheticMRI2D,coronal,wavelet,horizontal,71.0,71.0
5,medical,syntheticMRI2D,coronal,wavelet,vertical,71.0,71.0
6,medical,syntheticMRI2D,sagittal,wavelet,diagonal,71.0,71.0
7,medical,syntheticMRI2D,sagittal,wavelet,horizontal,71.0,86.0
8,medical,syntheticMRI2D,sagittal,wavelet,vertical,100.0,86.0
9,medical,syntheticMRI3D,full,wavelet,aad,67.0,67.0


In [ ]:

medical = mainDFGrouped.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="failure_category",
    values="number",
    aggfunc="sum",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
# Reorder columns if present in medical_pivot_percent
ordered_cols_present = [col for col in ordered_failcat_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent


failure_category                                            actually_pass  \
dataset_type dataset        subset   transform orientation                  
medical      syntheticMRI2D axial    wavelet   diagonal               0.0   
                                               horizontal             0.0   
                                               vertical               0.0   
                            coronal  wavelet   diagonal              29.0   
                                               horizontal             0.0   
                                               vertical               0.0   
                            sagittal wavelet   diagonal               0.0   
                                               horizontal             0.0   
                                               vertical               0.0   
             syntheticMRI3D full     wavelet   aad                    0.0   
                                               ada                    0.0   
                                               add                    0.0   
                                               daa                    0.0   
                                               dad                   20.0   
                                               dda                    0.0   
                                               ddd                   40.0   

failure_category                                            practically_pass  \
dataset_type dataset        subset   transform orientation                     
medical      syntheticMRI2D axial    wavelet   diagonal                 86.0   
                                               horizontal               71.0   
                                               vertical                 71.0   
                            coronal  wavelet   diagonal                 71.0   
                                               horizontal               71.0   
                                               vertical                 71.0   
                            sagittal wavelet   diagonal                 71.0   
                                               horizontal               71.0   
                                               vertical                100.0   
             syntheticMRI3D full     wavelet   aad                      67.0   
                                               ada                      67.0   
                                               add                      80.0   
                                               daa                      60.0   
                                               dad                      60.0   
                                               dda                     100.0   
                                               ddd                      60.0   

failure_category                                            interesting_failure  \
dataset_type dataset        subset   transform orientation                        
medical      syntheticMRI2D axial    wavelet   diagonal                     0.0   
                                               horizontal                   0.0   
                                               vertical                     0.0   
                            coronal  wavelet   diagonal                     0.0   
                                               horizontal                  14.0   
                                               vertical                     0.0   
                            sagittal wavelet   diagonal                     0.0   
                                               horizontal                  14.0   
                                               vertical                     0.0   
             syntheticMRI3D full     wavelet   aad                         17.0   
                                               ada                         17.0   
                                               add                         20.0   
          

In [ ]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "orientation"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                  GenGamma  \
dataset_type dataset        subset   transform orientation             
medical      syntheticMRI2D axial    wavelet   diagonal         62.0   
                                               horizontal       75.0   
                                               vertical         62.0   
                            coronal  wavelet   diagonal         75.0   
                                               horizontal       62.0   
                                               vertical         62.0   
                            sagittal wavelet   diagonal         62.0   
                                               horizontal       75.0   
                                               vertical         75.0   
             syntheticMRI3D full     wavelet   aad              57.0   
                                               ada              57.0   
                                               add              57.0   
                                               daa              57.0   
                                               dad              71.0   
                                               dda              86.0   
                                               ddd              86.0   

best_prior                                                  Gaussian  Laplace  \
dataset_type dataset        subset   transform orientation                      
medical      syntheticMRI2D axial    wavelet   diagonal         12.0     12.0   
                                               horizontal       25.0      0.0   
                                               vertical         38.0      0.0   
                            coronal  wavelet   diagonal          0.0      0.0   
                                               horizontal       25.0      0.0   
                                               vertical         25.0      0.0   
                            sagittal wavelet   diagonal         12.0      0.0   
                                               horizontal       12.0      0.0   
                                               vertical         12.0      0.0   
             syntheticMRI3D full     wavelet   aad              29.0      0.0   
                                               ada              14.0     14.0   
                                               add              14.0      0.0   
                                               daa              14.0     14.0   
                                               dad               0.0     14.0   
                                               dda               0.0      0.0   
                                               ddd               0.0      0.0   

best_prior                                                  Student-T  
dataset_type dataset        subset   transform orientation             
medical      syntheticMRI2D axial    wavelet   diagonal          12.0  
                                               horizontal         0.0  
                                               vertical           0.0  
                            coronal  wavelet   diagonal          12.0  
                                               horizontal         0.0  
                                               vertical           0.0  
                            sagittal wavelet   diagonal          12.0  
                                               horizontal         0.0  
                                               vertical           0.0  
             syntheticMRI3D full     wavelet   aad                0.0  
                                               ada                0.0  
                                               add               14.0  
                                               daa                0.0  
                                               dad                0.0  
                                               dda                0.0  
                    

In [ ]:

medical = main_df2.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical["number"] = 1
medical_pivot = medical.pivot_table(
    index=["dataset_type", "dataset", "subset", "transform", "failure_category"],
    columns="best_prior",
    values="number",
    aggfunc="count",
    fill_value=0,
)

# Calculate row sums for percentage calculation
row_sums = medical_pivot.sum(axis=1)
medical_pivot_percent = np.round(medical_pivot.div(row_sums, axis=0) * 100)

medical_pivot_percent
ordered_cols_present = [col for col in ordered_prior_cols if col in medical_pivot_percent.columns]
medical_pivot_percent = medical_pivot_percent[ordered_cols_present]
medical_pivot_percent

best_prior                                                          GenGamma  \
dataset_type dataset        subset   transform failure_category                
medical      syntheticMRI2D axial    wavelet   practically_pass         94.0   
                                               trivial_failure          12.0   
                            coronal  wavelet   actually_pass           100.0   
                                               interesting_failure       0.0   
                                               practically_pass         93.0   
                                               trivial_failure           0.0   
                            sagittal wavelet   interesting_failure     100.0   
                                               practically_pass         88.0   
                                               trivial_failure          17.0   
             syntheticMRI3D full     wavelet   actually_pass           100.0   
                                               interesting_failure       0.0   
                                               practically_pass        100.0   
                                               trivial_failure          25.0   

best_prior                                                          Gaussian  \
dataset_type dataset        subset   transform failure_category                
medical      syntheticMRI2D axial    wavelet   practically_pass          0.0   
                                               trivial_failure          75.0   
                            coronal  wavelet   actually_pass             0.0   
                                               interesting_failure     100.0   
                                               practically_pass          0.0   
                                               trivial_failure          50.0   
                            sagittal wavelet   interesting_failure       0.0   
                                               practically_pass          6.0   
                                               trivial_failure          33.0   
             syntheticMRI3D full     wavelet   actually_pass             0.0   
                                               interesting_failure      25.0   
                                               practically_pass          0.0   
                                               trivial_failure          25.0   

best_prior                                                          Laplace  \
dataset_type dataset        subset   transform failure_category               
medical      syntheticMRI2D axial    wavelet   practically_pass         0.0   
                                               trivial_failure         12.0   
                            coronal  wavelet   actually_pass            0.0   
                                               interesting_failure      0.0   
                                               practically_pass         0.0   
                                               trivial_failure          0.0   
                            sagittal wavelet   interesting_failure      0.0   
                                               practically_pass         0.0   
                                               trivial_failure          0.0   
             syntheticMRI3D full     wavelet   actually_pass            0.0   
                                               interesting_failure     50.0   
                                               practically_pass         0.0   
                                               trivial_failure          6.0   

best_prior                                                          Student-T  
dataset_type dataset        subset   transform failure_category                
medical      syntheticMRI2D axial    wavelet   practically_pass           6.0  
                                               trivial_failure            0.0  
                            coronal  wavelet   actually_pass              0.0  
            